# Addestramento 1

In [1]:
import tensorflow as tf
from keras.legacy_tf_layers.convolutional import Conv1D
from tensorflow import keras

In [2]:
model = keras.models.Sequential([
keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="valid", input_shape=[300, 1]),
keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="valid"),
keras.layers.MaxPooling1D(pool_size=2, strides=2),
keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, padding="valid"),
keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, padding="valid"),
keras.layers.MaxPooling1D(pool_size=2, strides=2),
keras.layers.Flatten(),
keras.layers.Dense(256, activation=tf.nn.relu),
keras.layers.Dense(128, activation=tf.nn.relu),
keras.layers.Dense(5, activation=tf.nn.softmax)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 296, 64)           384       
                                                                 
 conv1d_1 (Conv1D)           (None, 292, 64)           20544     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 146, 64)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 144, 128)          24704     
                                                                 
 conv1d_3 (Conv1D)           (None, 142, 128)          49280     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 71, 128)          0         
 1D)                                                    

In [3]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x=x_train, y=y_train, epochs=3)

test_error_rate = model.evaluate(x_test, y_test, verbose=0)

NameError: name 'x_train' is not defined

# Addestramento 2

In [4]:
import tensorflow as tf
!pip install d2l==1.0.0a1.post0
from d2l import tensorflow as d2

In [5]:
class Inception(tf.keras.Model):
     # `c1`--`c4` sono il numero di canali in output per ogni ramo
    def __init__(self, c1, c2, c3):
        super().__init__()
        self.b1_1 = tf.keras.layers.Conv2D(c1[0], 1, activation='relu')
        self.b1_2 = tf.keras.layers.MaxPool2D(3, 1, padding='same')
        self.b1_3 = tf.keras.layers.Conv2D(c1[0], 1, activation='relu')
        self.b2_1 = tf.keras.layers.Conv2D(c2[0], 1, activation='relu')
        self.b2_2 = tf.keras.layers.MaxPool2D(3, 1, padding='same')
        self.b3_1 = tf.keras.layers.Conv2D(c3[0], 1, activation='relu')
        self.b3_2 = tf.keras.layers.MaxPool2D(3, 1, padding='same')
        
    def call(self, x):
        b1 = self.b1_2(self.b1_1(x))
        b2 = self.b2_2(self.b2_1(x))
        b3 = self.b3_2(self.b3_1(x))
        return tf.keras.layers.Concatenate()([b1, b2, b3])

In [6]:
# example of creating a CNN with an efficient inception module
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Concatenate
from keras.utils import plot_model

# function for creating a projected inception module
# https://machinelearningmastery.com/how-to-implement-major-architecture-innovations-for-convolutional-neural-networks/
def inception_module(layer_in):
    # branch 1
    branch1 = Conv2D(8, (4,4), padding='same', activation='relu')(layer_in)
    branch1 = MaxPooling2D((2,2), strides=(1,1), padding='same')(branch1)
    branch1 = Conv2D(24, (6,6), padding='same', activation='relu')(branch1)
    branch1 = MaxPooling2D((2,2), strides=(1,1), padding='same')(branch1)
    # branch 2
    branch2 = Conv2D(8, (6,6), padding='same', activation='relu')(layer_in)
    branch2 = MaxPooling2D((2,2), strides=(1,1), padding='same')(branch2)
    branch2 = Conv2D(24, (8,8), padding='same', activation='relu')(branch2)
    branch2 = MaxPooling2D((2,2), strides=(1,1), padding='same')(branch2)
    # branch 3
    branch3 = Conv2D(8, (8,8), padding='same', activation='relu')(layer_in)
    branch3 = MaxPooling2D((2,2), strides=(1,1), padding='same')(branch3)
    branch3 = Conv2D(24, (10,10), padding='same', activation='relu')(branch3)
    branch3 = MaxPooling2D((2,2), strides=(1,1), padding='same')(branch3)
    # concatenate filters, assumes filters/channels last
    layer_out = Concatenate([branch1, branch2, branch3])
    return layer_out
 
# define model input
visible = Input(shape=(251, 1, 1))
# add inception block 1
layer = inception_module(visible)
print(type(layer))
# create model
model = tf.keras.models.Sequential([
    layer,
    Flatten(),
    Dense(256),
    Dense(32),
    Dense(4)])

model.build(visible)

print(model.summary())

<class 'keras.layers.merging.concatenate.Concatenate'>


TypeError: Cannot iterate over a Tensor with unknown first dimension.

In [20]:
# summarize model
model.summary()
# plot model architecture
plot_model(model, show_shapes=True, to_file='inception_module.png')

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

# a_ip_img = Input(shape=(32,32,1), name="Input_a")
input_tot = Input(shape=(251, 1, 1), name ="Input_tot")

# Pipeline 1
branch1_1 = Conv2D(8, (4,4), padding='same', activation='relu', name ="branch1_1")(input_tot)
branch1_2 = MaxPooling2D((2,2), strides=(1,1), padding='same', name ="branch1_2")(branch1_1)
branch1_3 = Conv2D(24, (6,6), padding='same', activation='relu', name ="branch1_3")(branch1_2)
branch1_4 = MaxPooling2D((2,2), strides=(1,1), padding='same', name ="branch1_4")(branch1_3)

# al_1 = Dense(64, activation = "relu",name ="a_layer_1")(a_ip_img)
# al_2 = Dense(128, activation="relu",name ="a_layer_2")(al_1)
# al_3 = Dense(64, activation="relu",name ="a_layer_3")(al_2)
# al_4 = Dense(32, activation="sigmoid",name ="a_output_layer")(al_3)

# Pipeline 2
branch2_1 = Conv2D(8, (6,6), padding='same', activation='relu', name ="branch2_1")(input_tot)
branch2_2 = MaxPooling2D((2,2), strides=(1,1), padding='same', name ="branch2_2")(branch2_1)
branch2_3 = Conv2D(24, (8,8), padding='same', activation='relu', name ="branch2_3")(branch2_2)
branch2_4 = MaxPooling2D((2,2), strides=(1,1), padding='same', name ="branch2_4")(branch2_3)

# bl_1 = Dense(64, activation="relu",name ="b_layer_1")(a_ip_img)
# bl_2 = Dense(32, activation = "sigmoid",name ="b_output_layer")(bl_1)

# Pipeline 3
branch3_1 = Conv2D(8, (8,8), padding='same', activation='relu', name ="branch3_1")(input_tot)
branch3_2 = MaxPooling2D((2,2), strides=(1,1), padding='same', name ="branch3_2")(branch3_1)
branch3_3 = Conv2D(24, (10,10), padding='same', activation='relu', name ="branch3_3")(branch3_2)
branch3_4 = MaxPooling2D((2,2), strides=(1,1), padding='same', name ="branch3_4")(branch3_3)

#Merging tre pipeline
# a_b = concatenate([al_4,bl_2],name="concatenated_layer")
branch_concatenate = concatenate([branch1_4,branch2_4,branch3_4],name="concatenated_layer")

#Final Layer
# output_layer = Dense(16, activation = "sigmoid", name = "output_layer")(a_b)

dense1 = Dense(256, activation = "sigmoid", name = "dense1")(branch_concatenate)
dense2 = Dense(32, activation = "sigmoid", name = "dense2")(dense1)
output_layer = Dense(4, activation = "sigmoid", name = "output_layer")(dense2)

#Model Definition
# merged = Model(inputs=[a_ip_img],outputs=[output_layer], name = "merged model")
merged = Model(inputs=[input_tot],outputs=[output_layer], name = "merged model")


#Model Details
merged.summary()
keras.utils.plot_model(merged, "output/architecture.png", show_shapes=True)

Model: "merged model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_tot (InputLayer)         [(None, 251, 1, 1)]  0           []                               
                                                                                                  
 branch1_1 (Conv2D)             (None, 251, 1, 8)    136         ['Input_tot[0][0]']              
                                                                                                  
 branch2_1 (Conv2D)             (None, 251, 1, 8)    296         ['Input_tot[0][0]']              
                                                                                                  
 branch3_1 (Conv2D)             (None, 251, 1, 8)    520         ['Input_tot[0][0]']              
                                                                                       

# Secondo modello funzionante

In [30]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

input_tot = Input(shape=(251, 1), name ="Input_tot")

# Pipeline 1
branch1_1 = keras.layers.Conv1D(filters=8, kernel_size=4, activation='relu', name ="branch1_1")(input_tot)
branch1_2 = keras.layers.MaxPooling1D(pool_size=2, strides=2, name ="branch1_2")(branch1_1)
branch1_3 = keras.layers.Conv1D(filters=24, kernel_size=6, activation='relu', name ="branch1_3")(branch1_2)
branch1_4 = keras.layers.MaxPooling1D(pool_size=2, strides=2, name ="branch1_4")(branch1_3)

# Pipeline 2
branch2_1 = keras.layers.Conv1D(filters=8, kernel_size=6, activation='relu', name ="branch2_1")(input_tot)
branch2_2 = keras.layers.MaxPooling1D(pool_size=2, strides=2, name ="branch2_2")(branch2_1)
branch2_3 = keras.layers.Conv1D(filters=24, kernel_size=8, activation='relu', name ="branch2_3")(branch2_2)
branch2_4 = keras.layers.MaxPooling1D(pool_size=2, strides=2, name ="branch2_4")(branch2_3)

# Pipeline 3
branch3_1 = keras.layers.Conv1D(filters=8, kernel_size=8, activation='relu', name ="branch3_1")(input_tot)
branch3_2 = keras.layers.MaxPooling1D(pool_size=2, strides=2, name ="branch3_2")(branch3_1)
branch3_3 = keras.layers.Conv1D(filters=24, kernel_size=10, activation='relu', name ="branch3_3")(branch3_2)
branch3_4 = keras.layers.MaxPooling1D(pool_size=2, strides=2, name ="branch3_4")(branch3_3)

#Merging tre pipeline
branch_concatenate = concatenate([branch1_4,branch2_4,branch3_4], axis=1, name="concatenated_layer")

#Final Layer
dense1 = Dense(256, activation = "sigmoid", name = "dense1")(branch_concatenate)
dense2 = Dense(32, activation = "sigmoid", name = "dense2")(dense1)
output_layer = Dense(4, activation = "sigmoid", name = "output_layer")(dense2)

#Model Definition
merged = Model(inputs=[input_tot],outputs=[output_layer], name = "merged model")

#Model Details
merged.summary()
keras.utils.plot_model(merged, "output/architecture.png", show_shapes=True)

Model: "merged model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_tot (InputLayer)         [(None, 251, 1)]     0           []                               
                                                                                                  
 branch1_1 (Conv1D)             (None, 248, 8)       40          ['Input_tot[0][0]']              
                                                                                                  
 branch2_1 (Conv1D)             (None, 246, 8)       56          ['Input_tot[0][0]']              
                                                                                                  
 branch3_1 (Conv1D)             (None, 244, 8)       72          ['Input_tot[0][0]']              
                                                                                       